In [1]:
from car_racing import CarRacing

import scipy.misc
import imageio
import numpy as np
import gym

import msvcrt
import win32api
import win32con

import h5py
from multiprocess import Process, Queue, Manager
import os

from random import choice, random, randint

env = CarRacing()


D:\Users\Kristopher_2\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
#Code written by Tielessin on https://stackoverflow.com/questions/46506850/how-can-i-get-input-from-an-xbox-one-controller-in-python
#Edited by Kristopher Karadimas

from inputs import get_gamepad
import math
import threading

class XboxController(object):
    MAX_TRIG_VAL = math.pow(2, 8)
    MAX_JOY_VAL = math.pow(2, 15)

    def __init__(self):

        self.LeftJoystickY = 0
        self.LeftJoystickX = 0
        self.RightJoystickY = 0
        self.RightJoystickX = 0
        self.LeftTrigger = 0
        self.RightTrigger = 0
        self.LeftBumper = 0
        self.RightBumper = 0
        self.A = 0
        self.X = 0
        self.Y = 0
        self.B = 0
        self.LeftThumb = 0
        self.RightThumb = 0
        self.Back = 0
        self.Start = 0
        self.LeftDPad = 0
        self.RightDPad = 0
        self.UpDPad = 0
        self.DownDPad = 0

        self._monitor_thread = threading.Thread(target=self._monitor_controller, args=())
        self._monitor_thread.daemon = True
        self._monitor_thread.start()


    def read(self): # return the buttons/triggers that you care about in this methode
        x = self.LeftJoystickX
        rt = self.RightTrigger
        lt = self.LeftTrigger
        return [x, rt, lt]


    def _monitor_controller(self):
        while True:
            events = get_gamepad()
            for event in events:
                if event.code == 'ABS_Y':
                    self.LeftJoystickY = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_X':
                    self.LeftJoystickX = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_RY':
                    self.RightJoystickY = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_RX':
                    self.RightJoystickX = event.state / XboxController.MAX_JOY_VAL # normalize between -1 and 1
                elif event.code == 'ABS_Z':
                    self.LeftTrigger = event.state / XboxController.MAX_TRIG_VAL # normalize between 0 and 1
                elif event.code == 'ABS_RZ':
                    self.RightTrigger = event.state / XboxController.MAX_TRIG_VAL # normalize between 0 and 1
                elif event.code == 'BTN_TL':
                    self.LeftBumper = event.state
                elif event.code == 'BTN_TR':
                    self.RightBumper = event.state
                elif event.code == 'BTN_SOUTH':
                    self.A = event.state
                elif event.code == 'BTN_NORTH':
                    self.X = event.state
                elif event.code == 'BTN_WEST':
                    self.Y = event.state
                elif event.code == 'BTN_EAST':
                    self.B = event.state
                elif event.code == 'BTN_THUMBL':
                    self.LeftThumb = event.state
                elif event.code == 'BTN_THUMBR':
                    self.RightThumb = event.state
                elif event.code == 'BTN_SELECT':
                    self.Back = event.state
                elif event.code == 'BTN_START':
                    self.Start = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY1':
                    self.LeftDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY2':
                    self.RightDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY3':
                    self.UpDPad = event.state
                elif event.code == 'BTN_TRIGGER_HAPPY4':
                    self.DownDPad = event.state





In [3]:
joy = XboxController()

def get_action():
        action = env.action_space.sample()
        inputs = joy.read()
        
        joystickinputfilter = 0 if (abs(inputs[0]) < 0.15) else inputs[0]
        joystickinputsign = -1 if (joystickinputfilter<0) else 1
        action[0] = joystickinputsign*abs((joystickinputfilter*0.65)**3)
        action[1] = min(inputs[1],0.8)**6
        action[2] = min(inputs[2],0.8)**6
        return action

In [4]:
def extractImages():
    h5f = h5py.File(f'D:/Kris\'s Workbench/FYP/data_image.h5','r')
    h5f2 = h5py.File(f'D:/Kris\'s Workbench/FYP/data_name.h5','r')

    i=0
    while(True):
        i+=1
        try:
            b = h5f[str(i)][:]
            b_data = h5f2[str(i)][()]
            imageio.imwrite(b_data, b)
        except:
            h5f.close()
            h5f2.close()
            os.remove(f'D:/Kris\'s Workbench/FYP/data_image.h5')
            os.remove(f'D:/Kris\'s Workbench/FYP/data_name.h5')
            break

In [17]:
directory = f'D:/Kris\'s Workbench/FYP/TrainingData/0/'

mynum=0

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename.endswith('.jpg'):
        name = (os.path.basename(f)).split('_')
        if int(name[1]) > mynum:
            mynum = int(name[0])+1
print(mynum)




obs = env.reset()
env.render()


#boolean take_screenshots decides whether screenshots will be saved
take_screenshots = True

if take_screenshots:
    h5f = h5py.File(f'D:/Kris\'s Workbench/FYP/data_image.h5','w')
    h5f2 = h5py.File(f'D:/Kris\'s Workbench/FYP/data_name.h5','w')

t = 0
r = 0




#while(True):
for t in range(1500):
    t+=1
    action = get_action()
    obs, reward, done, _ = env.step(action)
    env.render()
    r += reward
    
    eps=1
    action_0 = action[0]
    action_1 = action[1]
    action_2 = action[2]
    
    if take_screenshots:
        h5f.create_dataset(str(t),data=obs)
        h5f2.create_dataset(str(t),data=f'D:/Kris\'s Workbench/FYP/TrainingData/0/{mynum}_{t}_{action_0}_{action_1}_{action_2}_{reward}.jpg')

if take_screenshots:
    h5f.close()
    h5f2.close()
    extractImages()
env.close()

5
Track generation: 1050..1317 -> 267-tiles track
